---
# **1. Setup and Initialization**
---

In [4]:
import ee
import geemap

In [5]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-group1project')

# Create a map object
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

---
# **2. Define Study Area and Load Imagery**
---

In [6]:
# Define the study area using a 5 km buffer around the point
# coordinates for project (-97.733330, 30.266666)
point = ee.Geometry.Point([-97.733330, 30.266666])
buffer_distance = 5000  # in meters
study_area = point.buffer(buffer_distance).bounds()

In [7]:
# Load Landsat 8 imagery and create a composite
image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(study_area)
    .filterDate("2016-01-01", "2016-12-31")
    .filter(ee.Filter.lt("CLOUD_COVER", 20))  # Filter images with <20% cloud cover
    .median()  # Composite to reduce noise
    .select("SR_B[1-7]")
)

In [8]:
# Visualization parameters
vis_params = {"min": 0, "max": 30000, "bands": ["SR_B5", "SR_B4", "SR_B3"]}
Map.centerObject(study_area, 10)
Map.addLayer(image, vis_params, "Landsat-8 Composite")
Map

Map(center=[30.266671164139705, -97.73324529505567], controls=(WidgetControl(options=['position', 'transparent…

---
# **3. Load NLCD Data and Generate Training Points**
---

In [9]:
# Load NLCD 2016 data and clip to the study area
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(study_area)
Map.addLayer(nlcd, {}, "NLCD")

In [10]:
# Stratified sampling for balanced class representation
points = nlcd.stratifiedSample(
    **{
        "numPoints": 1000,  # Reduced sample size
        "classBand": "landcover",
        "region": study_area,
        "scale": 30,
        "seed": 42,
        "geometries": True,
    }
)
Map.addLayer(points, {}, "Stratified Training Points")

print(points.size().getInfo())
print(points.first().getInfo())

11019
{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-97.72557159348098, 30.246210972829083]}, 'id': '0', 'properties': {'landcover': 11}}


---
# **4. Add Spectral Indices for Feature Enhancement**
---

In [11]:
# Define spectral bands
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]

In [12]:
# Add NDVI and NDWI
# NDVI and NDWI are spectral indices that are used to extract information from satellite images
ndvi = image.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")
ndwi = image.normalizedDifference(["SR_B3", "SR_B5"]).rename("NDWI")
image = image.addBands([ndvi, ndwi])
bands.extend(["NDVI", "NDWI"])

---
# **5. Train SVM Classifier**
---

In [13]:
# Support Vector Machines (SVMs) are machine learning algorithms that classify data by finding a hyperplane that separates data points into different classes
# Label for land cover classes
label = "landcover"

# Create a training dataset
training = image.select(bands).sampleRegions(
    **{
        "collection": points,
        "properties": [label],
        "scale": 30,
    }
)


In [14]:
# Train the Random Forest classifier
trained = ee.Classifier.smileRandomForest(50).train(
    features=training, classProperty=label, inputProperties=bands
)

# Print the first feature of the training data to verify
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'NDVI': 0.004994283430278301, 'NDWI': 0.023845704272389412, 'SR_B1': 8061, 'SR_B2': 8197, 'SR_B3': 8759, 'SR_B4': 8268, 'SR_B5': 8351, 'SR_B6': 8063, 'SR_B7': 7777, 'landcover': 11}}


In [15]:
# Classify the image using the trained classifier
result = image.select(bands).classify(trained)
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(center=[30.266671164139705, -97.73324529505567], controls=(WidgetControl(options=['position', 'transparent…

In [16]:
class_values = nlcd.get("landcover_class_values").getInfo()
class_values
class_palette = nlcd.get("landcover_class_palette").getInfo()
class_palette
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

Map.addLayer(landcover, {}, "Land cover")
Map

Map(center=[30.266671164139705, -97.73324529505567], controls=(WidgetControl(options=['position', 'transparent…

In [17]:
print("Change layer opacity:")

cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))
Map.add_legend(builtin_legend="NLCD")
Map

Change layer opacity:


Map(center=[30.266671164139705, -97.73324529505567], controls=(WidgetControl(options=['position', 'transparent…

---
# **6. Validate Classifier and Calculate Metrics**
---

In [18]:
# Generate validation points
validation_points = nlcd.stratifiedSample(
    **{
        "numPoints": 500,  # Reduced validation sample size
        "classBand": "landcover",
        "region": study_area,
        "scale": 30,
        "seed": 84,
        "geometries": True,
    }
)


In [19]:
# Create validation dataset by sampling regions
validation = image.select(bands).sampleRegions(
    collection=validation_points,
    properties=[label],
    scale=30,
)

In [20]:
# Classify validation points
validated = validation.classify(trained)

# Printing the Accuracy's Output
confusion_matrix = validated.errorMatrix(label, "classification")
print("Overall Accuracy:", confusion_matrix.accuracy().getInfo())
print("Kappa Coefficient:", confusion_matrix.kappa().getInfo())

Overall Accuracy: 0.7353890862124637
Kappa Coefficient: 0.7137741196579307


In [21]:
# Generate confusion matrix
import pandas as pd

# Assuming confusion_matrix is already fetched and stored as a nested list
matrix = confusion_matrix.getInfo()  # Replace this with your confusion matrix data

# Check the number of classes in the matrix
print("Confusion Matrix Dimensions:", len(matrix), len(matrix[0]))

# Generate generic labels if class_values does not match
class_labels = class_values if len(class_values) == len(matrix) else [f"Class {i}" for i in range(len(matrix))]

# Convert the confusion matrix to a pandas DataFrame
confusion_df = pd.DataFrame(matrix, index=class_labels, columns=class_labels)

# Display the confusion matrix as a table
print("Confusion Matrix:")
print(confusion_df)

# Optionally, save it as a CSV file for detailed analysis
confusion_df.to_csv("confusion_matrix.csv", index=True)

Confusion Matrix Dimensions: 96 96
Confusion Matrix:
          Class 0  Class 1  Class 2  Class 3  Class 4  Class 5  Class 6  \
Class 0         0        0        0        0        0        0        0   
Class 1         0        0        0        0        0        0        0   
Class 2         0        0        0        0        0        0        0   
Class 3         0        0        0        0        0        0        0   
Class 4         0        0        0        0        0        0        0   
...           ...      ...      ...      ...      ...      ...      ...   
Class 91        0        0        0        0        0        0        0   
Class 92        0        0        0        0        0        0        0   
Class 93        0        0        0        0        0        0        0   
Class 94        0        0        0        0        0        0        0   
Class 95        0        0        0        0        0        0        0   

          Class 7  Class 8  Class 9  ...  Clas